In [19]:
from resmem import ResMem, transformer
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import pathlib as Path
import numpy as np
import seaborn as sns 
import pandas as pd
from scipy import stats

In [2]:
model = ResMem(pretrained=True)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [34]:
df = pd.read_csv('target_and_distractor_images.csv')
df.replace('.pkl','.jpg',regex=True,inplace=True)

In [39]:
PATH_TO_ALL_IMAGES=Path.Path('../task/pilot_images/')


images_list=[]
layer_list=[]
value_list=[]

for col in df.columns:
    cur_images=df[col].values
    for curr_img in cur_images: 
        img = Image.open(PATH_TO_ALL_IMAGES /curr_img) 
        img.convert('RGB')
        # This will convert your image into RGB, for instance if it's a PNG (RGBA) or if it's black and white.

        model.eval()
        # Set the model to inference mode.

        image_x = transformer(img)
        # Run the preprocessing function
        
        # For a single image, the image must be reshaped into a batch
        # with size 1.
        # Get your prediction!
        prediction = model(image_x.view(-1, 3, 227, 227))
        prediction = prediction.data[0][0]
        
        #predictions[name] = prediction.item()
        value_list.append(prediction.item())
        images_list.append(curr_img)
        layer_list.append(col)

In [40]:
images_memorability_df=pd.DataFrame({'img_name':images_list,'layer':layer_list,'mem_value':value_list})

In [42]:
images_memorability_df.groupby('layer').aggregate({'mem_value':['mean','std']})

mem_value          
                  mean       std
layer                           
layer1        0.797212  0.061006
layer2        0.814454  0.074836
layer3        0.801848  0.068229
target_image  0.823045  0.064900